In [1]:
import ITensors:op
import Pkg; Pkg.add("HDF5")
import Pkg; Pkg.add("Plots")
import Pkg; Pkg.add("PyFormattedStrings")
import Pkg; Pkg.add("LinearAlgebra")
import Pkg; Pkg.add("LaTeXStrings")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
using ITensors, ITensorMPS
import ITensors:op
using HDF5
using Plots
using PyFormattedStrings
using LinearAlgebra

ITensors.disable_warn_order()

14

In [3]:
#include("overload_Spin=3h.jl")
# overloadings for SiteType 3/2
#----------------------------------------------
ITensors.space(::SiteType"S=3/2") = 4

# operator Sz
#-------------------
ITensors.op(::OpName"Sz",::SiteType"S=3/2") =
[+3/2   0    0    0
  0    +1/2  0    0
  0     0   -1/2  0
  0     0    0   -3/2]

# operator S+
#-------------------
ITensors.op(::OpName"S+",::SiteType"S=3/2") =
[0  √3  0   0 
 0  0   √4  0
 0  0   0   √3
 0  0   0   0]

# operator S-
#-------------------
ITensors.op(::OpName"S-",::SiteType"S=3/2") =
[0  0   0   0
 √3 0   0   0
 0  √4  0   0
 0  0   √3  0]

# z projection states
#-----------------------------------------
ITensors.state(::StateName"+3/2",::SiteType"S=3/2") = [1, 0, 0, 0]
ITensors.state(::StateName"+1/2",::SiteType"S=3/2") = [0, 1, 0, 0]
ITensors.state(::StateName"-1/2",::SiteType"S=3/2") = [0, 0, 1, 0]
ITensors.state(::StateName"-3/2",::SiteType"S=3/2") = [0, 0, 0, 1]

In [4]:
# construction of the MPO hamiltonian
#------------------------------------------
function Hamiltonian(sites)
    ops = OpSum()

    N = length(sites)
    # spin-spin interaction
    for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)
    #--------------------------
    return H
end

Hamiltonian (generic function with 1 method)

In [5]:

#four point RDM of sites AB, CD
# AB adjacent, CD adjacent
#------------------------------------------
function get_4pdm(wf::MPS,locs)
    psi = copy(wf)
    A,C = locs

    orthogonalize!(psi,A)
    ket = psi[A]
    for k in A+1:C+1
        ket *= psi[k]
    end
    rho = prime(ket,"Site") * dag(ket)

    if C-A>2
        inds_list = inds(rho)
        trinds1 = collect(3 : C-A)
        trinds2 = trinds1[end] .+ trinds1 .+ 2
        for idx in 1:length(trinds1)
            rho = rho*delta(inds_list[trinds1[idx]],inds_list[trinds2[idx]])
        end
    end
    #-------------------
    return rho
end

get_4pdm (generic function with 1 method)

In [6]:
#   calc_Eigenvalues of density matrix
#-----------------------------------
function calc_Eigenvalues(density_matrix)
    egn_val,_ = eigen(density_matrix,ishermitian=true)
    #-----------------------------------
    return diag(array(egn_val))
end


#   eigenvalue based generic properties of a density matrix
#-----------------------------------
function calc_Norm(density_matrix)
    rho = copy(density_matrix)

    egn_val = calc_Eigenvalues(rho)
    Norm = sum(egn_val)
    Purity = sum(egn_val.^2)

    #-----------------------------------
    return [Norm,Purity]
end

#   eigenvalue based generic properties of a density matrix
#-----------------------------------
function calc_SvN(density_matrix)
    rho = copy(density_matrix)

    egn_val = calc_Eigenvalues(rho)
    SvN = sum( [ - lam*log(lam) for lam in egn_val if lam > 0 ] )

    #-----------------------------------
    return SvN
end


calc_SvN (generic function with 1 method)

In [7]:
#   entanglement details between four sites
#-----------------------------------
function calc_LogNeg_4site(wf::MPS,locs)    
    psi = copy(wf)
    A,C = locs

    rho = get_4pdm(psi,[A,C])
    old = inds(rho)

    function get_LogNeg(rho,old,new)
        rho_PT = swapinds(rho,old,new)
        egn_val = calc_Eigenvalues(rho_PT)
        Neg =  abs(sum( [lam for lam in egn_val if lam<0] ))
        return log(1+2*Neg)
    end
        
    #   ABCD = 1234 
    #   ABCD = 5678

    # wrt A B C D
    new_A  = [old[5],old[2],old[3],old[4],old[1],old[6],old[7],old[8]]
    new_B  = [old[1],old[6],old[3],old[4],old[5],old[2],old[7],old[8]]
    new_C  = [old[1],old[2],old[7],old[4],old[5],old[6],old[3],old[8]]
    new_D  = [old[1],old[2],old[3],old[8],old[5],old[6],old[7],old[4]]

    # wrt AB AC AD
    new_AB = [old[5],old[6],old[3],old[4],old[1],old[2],old[7],old[8]]
    new_AC = [old[5],old[2],old[7],old[4],old[1],old[6],old[3],old[8]]
    new_AD = [old[5],old[2],old[3],old[8],old[1],old[6],old[7],old[4]]

    LogNeg = 1
    for new in [new_A,new_B,new_C,new_D,new_AB,new_AC,new_AD]
        LogNeg *= get_LogNeg(rho,old,new)
    end
    LogNeg = LogNeg^(1.0/7.0)

    #-----------------------------------
    return LogNeg
end

calc_LogNeg_4site (generic function with 1 method)

In [8]:
#define the parameters of the hamiltonian and constuct MPO
#----------------------------------------------------------------------------
J = 1 
B = 0.01

nsweeps = 20
maxdim = [10,25,50,100,150,200,250,300,400,500]
cutoff = [1E-11]

#--------------------
N = 30
#--------------------

site_list = collect(1:N) 
sites = siteinds( k->isodd(k) ? "S=1/2" : "S=1", N ) 
#sites = siteinds("S=1/2", N)

H = Hamiltonian(sites) 

# ground state and first excited state
#--------------------------------------------
energy0,psi0 = dmrg(H,random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true) ;
#energy1,psi1 = dmrg(H,[psi0],random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true) ;

After sweep 1 energy=-21.080896727766984  maxlinkdim=8 maxerr=2.68E-16 time=19.698
After sweep 2 energy=-21.510530465781237  maxlinkdim=25 maxerr=1.35E-07 time=0.121
After sweep 3 energy=-21.819419786795663  maxlinkdim=50 maxerr=4.83E-07 time=0.769
After sweep 4 energy=-21.854436026741713  maxlinkdim=100 maxerr=4.14E-08 time=2.779
After sweep 5 energy=-21.864856010717666  maxlinkdim=150 maxerr=7.70E-09 time=5.990
After sweep 6 energy=-21.869926255826616  maxlinkdim=200 maxerr=1.32E-09 time=11.033
After sweep 7 energy=-21.872666877274085  maxlinkdim=250 maxerr=3.06E-10 time=19.627
After sweep 8 energy=-21.874727713518453  maxlinkdim=300 maxerr=7.96E-11 time=28.682
After sweep 9 energy=-21.876782393294768  maxlinkdim=400 maxerr=1.08E-11 time=46.083
After sweep 10 energy=-21.87910435553384  maxlinkdim=433 maxerr=1.00E-11 time=53.353
After sweep 11 energy=-21.88151806505273  maxlinkdim=445 maxerr=9.99E-12 time=56.896
After sweep 12 energy=-21.883421884348117  maxlinkdim=440 maxerr=1.00E-11

In [9]:
#define the parameters of the hamiltonian and constuct MPO
#----------------------------------------------------------------------------

J = 1 
B = 0.01

nsweeps = 20
maxdim = [10,25,50,100,150,200,250,300,400,500]
cutoff = [1E-11]

#--------------------
N = 30
#--------------------

site_list = collect(1:N) 

sites = siteinds("S=1/2", N)

H1 = Hamiltonian(sites) 

# ground state and first excited state
#--------------------------------------------
energy0,psi01 = dmrg(H1,random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true) ;
#energy1,psi1 = dmrg(H,[psi0],random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true) ;

After sweep 1 energy=-12.889552561008202  maxlinkdim=4 maxerr=1.67E-15 time=0.036
After sweep 2 energy=-13.119415329316666  maxlinkdim=16 maxerr=3.82E-16 time=0.118
After sweep 3 energy=-13.320484915509596  maxlinkdim=50 maxerr=9.48E-07 time=0.345
After sweep 4 energy=-13.321951265255937  maxlinkdim=100 maxerr=2.28E-07 time=1.611
After sweep 5 energy=-13.3219620845331  maxlinkdim=150 maxerr=2.47E-08 time=3.999
After sweep 6 energy=-13.321962909244542  maxlinkdim=200 maxerr=4.02E-09 time=5.675
After sweep 7 energy=-13.321963030819676  maxlinkdim=250 maxerr=8.00E-10 time=9.542
After sweep 8 energy=-13.321963052305303  maxlinkdim=300 maxerr=1.82E-10 time=14.331
After sweep 9 energy=-13.321963057898841  maxlinkdim=400 maxerr=1.79E-11 time=21.798
After sweep 10 energy=-13.321963058062043  maxlinkdim=430 maxerr=1.00E-11 time=25.754
After sweep 11 energy=-13.321963058064146  maxlinkdim=431 maxerr=9.99E-12 time=25.353
After sweep 12 energy=-13.321963058064139  maxlinkdim=431 maxerr=9.99E-12 ti

In [10]:
using Plots
using LaTeXStrings

seps=0:5
enta= []
enta1= []
A = 1
    println("Mixed spin chain 1/2-3/2")
for sep in seps
    D =  A+2+sep
    ent = calc_LogNeg_4site(psi0,[A,D])
    push!(enta, ent)
    println(f"{sep} \t {ent}")
end
println("----------------------------")
println("----------------------------")
    println("Spin-half")
for sep in seps
    D =  A+2+sep
    ent1 = calc_LogNeg_4site(psi01,[A,D])
    push!(enta1, ent1)
    println(f"{sep} \t {ent1}")
end

plot(seps, enta, xlabel= "Separation", ylabel=L"\mathcal{N}_4", label="1/2--1", marker=:circle, color="red", lw=2)
plot!(seps, enta1, xlabel= "Separation", ylabel=L"\mathcal{N}_4",label="1/2--1/2", linestyle=:dash, marker=:circle, color="blue", lw=2)

Mixed spin chain 1/2-3/2
0 	 0.5548891940190653
1 	 0.28060257362439656
2 	 0.1272163369338806
3 	 0.04378065951295012
4 	 0.0
5 	 0.0
----------------------------
----------------------------
Spin-half
0 	 0.5321493724052067
1 	 0.26864128629138434
2 	 0.0
3 	 0.0
4 	 0.0
5 	 0.0


GKS: dlopen(/Users/Hank/.julia/artifacts/66ebf8a541da7166677a6e2ac6b8277592d32044/lib/cairoplugin.so, 1): Library not loaded: @rpath/libcairo.2.dylib
  Referenced from: /Users/Hank/.julia/artifacts/66ebf8a541da7166677a6e2ac6b8277592d32044/lib/cairoplugin.so
  Reason: Incompatible library version: cairoplugin.so requires version 11603.0.0 or later, but libcairo.2.dylib provides version 2.0.0


In [11]:
#A = 1
#D = 5
#rho1 = get_4pdm(psi0,[A,D])
#d0, u0 = eigen(rho1,ishermitian=true) 
#@show diag(array(d0))